# AutoParallel

AutoParallel, a Python module to automatically find an appropriate task-based parallelization of affine loop nests to execute them in parallel in a distributed computing infrastructure. This parallelization can also include the building of data blocks to increase task granularity in order to achieve a good execution performance. Moreover, AutoParallel is based on sequential programming and only contains a small annotation in the form of a Python decorator so that anyone with little programming skills can scale up an application to hundreds of cores. 


The implementation includes:
* The `@parallel()` decorator for Python to run applications with [PyCOMPSs][compss]
* Different Translator modules to automatically generate parallel Python code
    * Py2Scop Translator: Translation of Python code into [OpenScop][openscop] format 
    * Scop2PScop2Py Translator: Integration with the [PLUTO][pluto] tool to generate
    possible parallelizations of the given code. The output is written in Python
    (by means of a [CLooG][cloog] extension) with parallel annotations following an
    OMP fashion. 
    * Py2PyCOMPSs Translator: Translation of parallel annotated Python code into
    PyCOMPSs task definitions. 
 * A Code Replacer module to replace the user code by autogenerated code.
 * Several example applications (`examples/` folder).
 
[compss]: http://compss.bsc.es
[pluto]: http://pluto-compiler.sourceforge.net/
[openscop]: http://icps.u-strasbg.fr/people/bastoul/public_html/development/openscop/
[cloog]: http://www.cloog.org/

# Architecture
Next, we describe the five components of the AutoParallel module. For the sake of clarity, The Figure shows the relationship between all the components and its expected inputs and outputs. 

![AutoParallel Architecture](./figures/autoparallel_internals.jpeg "AutoParallel Architecture")

* **Decorator**: Implements the `@parallel()` decorator to detect functions that the user has marked as potentially parallel.
* **Python To OpenScop Translator**: For each affine loop nest detected in the user function, builds a Python Scop object representing it that can be bulked into an OpenScop format file.
* **Parallelizer**: Returns the Python code resulting from parallelizing an OpenScop file. Since Python does not have any standard regarding parallel annotations, the parallel loops are annotated using comments with OpenMP syntax.
* **Python to PyCOMPSs Translator**: Converts an annotated Python code into a PyCOMPSs application by inserting the necessary task annotations and data synchronizations. This component also performs loop taskification if it is enabled. 
* **Code Replacer**: Replaces each loop nest in the initial user code by the auto-generated code so that PyCOMPSs can execute the code in a distributed computing platform. When the application has finished, it restores the user code and saves the auto-generated code in a separated file.


# Examples

## Cholesky
### Description
The Cholesky factorization can be applied to Hermitian positive-defined matrices. This decomposition is a particular case of the LU factorization, obtaining two matrices of the form $U = L^t$. Our version of this application applies the right-looking algorithm because it is more aggressive, meaning that in an early stage of the computation there are blocks of the solution that are already computed and all the potential parallelism is released as soon as possible. 

### User code

In [ ]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
import IPython

with open('examples/cholesky/cholesky.py') as f:
    code = f.read()

formatter = HtmlFormatter()
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(
    formatter.get_style_defs('.highlight'),
    highlight(code, PythonLexer(), formatter)))

### Execution Setup

In [ ]:
%%bash

# Store application properties
APP_NAME="cholesky"
APP_PROPERTIES="$(pwd)/examples/${APP_NAME}/exec/exec.properties"

# Create exec folder
mkdir -p "$(pwd)/examples/${APP_NAME}/exec"

cat > "${APP_PROPERTIES}" << EOT
APP_NAME=${APP_NAME}
APP_DIR="$(pwd)/examples/${APP_NAME}"
EXEC_DIR="$(pwd)/examples/${APP_NAME}/exec"
MONITOR_DIR="$(pwd)/examples/${APP_NAME}/exec/monitor"
TRACE_DIR="$(pwd)/examples/${APP_NAME}/exec/trace"
OUT_FILE="$(pwd)/examples/${APP_NAME}/exec/out.log"
ERROR_FILE="$(pwd)/examples/${APP_NAME}/exec/error.log"
XMLS_DIR="$(pwd)/examples/xml"
EOT


### Execution

In [ ]:
%%bash

# Application parameters
MSIZE=4
BSIZE=4

# Application properties
APP_NAME="cholesky"
APP_PROPERTIES="$(pwd)/examples/${APP_NAME}/exec/exec.properties"
source "${APP_PROPERTIES}"

# Task Cores
export ComputingUnits=1

# Run application
runcompss \
  -d \
  -tg \
  --summary \
  --lang=python \
  --project="${XMLS_DIR}/project.xml" \
  --resources="${XMLS_DIR}/resources.xml" \
  --pythonpath="${APP_DIR}" \
  --specific_log_dir="${EXEC_DIR}" \
  "${APP_DIR}/${APP_NAME}".py $MSIZE $BSIZE > >(tee "${EXEC_DIR}/out.log") 2> >(tee "${EXEC_DIR}/error.log" >&2)

### Parsed Output

In [ ]:
%%bash

# Application properties
APP_NAME="cholesky"
APP_PROPERTIES="$(pwd)/examples/${APP_NAME}/exec/exec.properties"
source "${APP_PROPERTIES}"

# Parse results
grep -A 8 "RESULTS -----------------" "${OUT_FILE}"
grep -A 7 "COMPSs Task Execution Summary --" "${OUT_FILE}"

### Generated Task Graph

In [ ]:
%%bash

# Application properties
APP_NAME="cholesky"
APP_PROPERTIES="$(pwd)/examples/${APP_NAME}/exec/exec.properties"
source "${APP_PROPERTIES}"

# Generate PNG graph file
dot -Tpng -Gnewrank=true "${MONITOR_DIR}"/complete_graph.dot > "${MONITOR_DIR}"/complete_graph.png

![Cholesky Task Graph](./examples/cholesky/exec/monitor/complete_graph.png "Cholesky Task Graph")

### Generated Trace

In [ ]:
%%bash

# Application properties
APP_NAME="cholesky"
APP_PROPERTIES="$(pwd)/examples/${APP_NAME}/exec/exec.properties"
source "${APP_PROPERTIES}"

# Display trace
wxparaver ${TRACE_DIR}/*.prv

### Generated code

In [ ]:
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import HtmlFormatter
import IPython

with open('examples/cholesky/cholesky_autogen.py') as f:
    code = f.read()

formatter = HtmlFormatter()
IPython.display.HTML('<style type="text/css">{}</style>{}'.format(
    formatter.get_style_defs('.highlight'),
    highlight(code, PythonLexer(), formatter)))

### Clean execution files

In [ ]:
%%bash

# Application properties
APP_NAME="cholesky"
APP_PROPERTIES="$(pwd)/examples/${APP_NAME}/exec/exec.properties"
source "${APP_PROPERTIES}"

# Recover application code if something failed
if [ -f "${APP_DIR}/${APP_NAME}_bkp.py" ]; then
  mv "${APP_DIR}/${APP_NAME}_bkp.py" "${APP_DIR}/${APP_NAME}.py"
fi

# Clean generated files
rm -rf "${EXEC_DIR}"
rm -f "${APP_DIR}/${APP_NAME}.pyc"
rm -f "${APP_DIR}/${APP_NAME}_autogen.py"
rm -f "${APP_PROPERTIES}"